In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pandasql import sqldf
from sklearn import preprocessing
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 700)

with open('output.csv') as f:
    print(f)

    
deferments = pd.read_csv("output.csv", sep = ";", encoding ="unicode_escape")
design = pd.read_csv("Designs_alle.csv", sep = ";")
tiefpumpen = pd.read_csv("Tiefpumpen.csv", sep = ";")
data = pd.read_csv("Designs_alle1.csv",  sep = ";", thousands=',')
runlife = pd.read_csv("RUNLIFE_final1.csv", sep = ";")

data_design = data.drop(data.loc[:,'TYP':'D#KUM_LASTWECHSEL_LAST_CALC'], axis = 1)
data_design = data_design.drop_duplicates()

data_tp = data.drop(data.loc[:, 'PK#ID_1':'DYNO_AUFNAHME'], axis = 1)
data_tp = data_tp.drop(columns = ['PK#ID','FARBCODE','DATUM','BEARBEITER','BEARBEITER#X_KEY','AUSFALLDATUM','BEHANDLUNGSGRUND'])
data_tp = data_tp.drop(data_tp.loc[:, 'FAE3':'FLG_FREIGABE'], axis = 1)
data_tp = data_tp.drop_duplicates()

data_raw = sqldf("SELECT runlife.*, data_tp.* \
      FROM runlife JOIN data_tp ON runlife.tp_nummer = data_tp.tiefpumpennummer \
      AND runlife.sondenkurzname_1 = data_tp.sondenkurzname")

data = sqldf("SELECT data_raw.*, data_design.* \
      FROM data_raw JOIN data_design ON data_raw.datum_1 = data_design.datum_1 \
      AND data_raw.sondenkurzname_1 = data_design.sondenkurzname_1")

data['KUM_ARBEITSTAGE'] = data['KUM_ARBEITSTAGE'].astype(float)

data = sqldf("SELECT * FROM data WHERE runlife > 0 AND runlife IS NOT NULL").drop_duplicates()

data_deferments = sqldf("SELECT data.*, deferments.* FROM data JOIN deferments ON data.tp_nummer = deferments.tpnummer")
deta_deferments = data_deferments.drop_duplicates()

deta_deferments

In [ ]:
sqldf("SELECT level4, count(level4) FROM data_deferments WHERE abstellgrund LIKE '%Installation undicht%' GROUP BY level4 ORDER BY \
count(level4) DESC")

In [ ]:
sqldf("SELECT level4, count(level4) FROM data_deferments WHERE level3a LIKE '%Valve rod%' GROUP BY level4 ORDER BY \
count(level4) DESC")

In [ ]:
datum = pd.to_datetime(data['DATUM_1'], format="%m.%d.%Y %H:%M:%S")
pumpdurchmesser = data['API_KURZBEZ_TEIL1'].str[3:]
pumpentyp = pd.get_dummies(data['API_KURZBEZ_TEIL2'])
pgzustand = pd.get_dummies(data['PG_TYP2'])
tubingcondition = pd.get_dummies(data['TUB_TYP2'])
gasanker = pd.get_dummies(data['GAS_ANKER'])
packer = pd.get_dummies(data['PACK_ANKER'])
hubzahl = data['HUBZAHL']
hublaenge = data['HUBLAENGE']
farr = data['SPEZ_FARR'].fillna(0).astype(int)
spez1 = pd.get_dummies(data['SPEZ_EQUI1'])
spez2 = pd.get_dummies(data['SPEZ_EQUI2'])
spez3 = pd.get_dummies(data['SPEZ_EQUI3'])
spez4 = pd.get_dummies(data['SPEZ_EQUI4'])
spez5 = pd.get_dummies(data['SPEZ_EQUI5'])
spez6 = pd.get_dummies(data['SPEZ_EQUI6'])

workdata = pd.concat([data['SONDENKURZNAME_1'], datum,
                      data['RUNLIFE'],data['DOGLEG'],
                      data['ABSETZTEUFE'],data['HUBZAHL'],
                      data['HUBLAENGE'], hublaenge, pumpdurchmesser,
                      pumpentyp, pgzustand, tubingcondition, gasanker, packer,
                      spez1, spez2, spez3, spez4, spez5, spez6, tubingcondition], axis = 1)

features_grouped = workdata.groupby(workdata.columns, axis=1).sum()

farr_heavycage_brass = features_grouped['TP mit FARR Pl., Messing-Barrel und schweren Kugeln & Sitzen ausstatten']
heavycage = features_grouped['!!OLD!!DONT USE!! TP mit schweren Kugeln und Sitzen ausstatten']

features_grouped['Tubing_pump'] = features_grouped['TH']
features_grouped['Sand_filter'] = features_grouped['Conslot Filter'] + features_grouped['Stanley Filter']
features_grouped['New_tubing'] = features_grouped['neu'] + features_grouped['gelb']
features_grouped['Spiral_rod_guides'] = features_grouped['Spiral Rod Guide 2 7/8"'] + features_grouped['Spiral Rod Guide 3 1/2"']
features_grouped['Heavy_wall_barrel'] = features_grouped['RHAC'] + features_grouped['RHBM'] + features_grouped['TH']
features_grouped['Jacket_pipe'] = features_grouped['Mantelrohr 2 7/8"'] + features_grouped['Mantelrohr 3 1/2"']
features_grouped['Gas_separator'] = features_grouped['Gasanker'] + features_grouped['Gasseparator - PCM']
features_grouped['Brass_barrel'] = features_grouped['Messingbarrel'] + farr_heavycage_brass
features_grouped['FARR_plunger'] = farr + farr_heavycage_brass
features_grouped['Heavy_cages_balls'] = heavycage + farr_heavycage_brass
features_grouped['Tubing_anchored'] = features_grouped['Steigrohranker'] + features_grouped['LSP ohne Gummi']
features_grouped['A4_valve_balls'] = np.sign(features_grouped['A4 Ventilkugeln'] + \
                                             features_grouped['Heavy_cages_balls'])

features_grouped['A4_valve_balls'] = np.sign(features_grouped['A4 Ventilkugeln'])
features_grouped['A6_valve_balls'] = np.sign(features_grouped['A6 Ventilkugeln'])
features_grouped['Sand_filter'] = np.sign(features_grouped['Sand_filter'].astype(int))
features_grouped['New_tubing'] = np.sign(features_grouped['New_tubing'])
features_grouped['Spiral_rod_guides'] = np.sign(features_grouped['Spiral_rod_guides'])
features_grouped['Heavy_wall_barrel'] = np.sign(features_grouped['Heavy_wall_barrel'].astype(int))
features_grouped['Gas_separator'] = np.sign(features_grouped['Gas_separator'].astype(int))
features_grouped['Brass_barrel'] = np.sign(features_grouped['Brass_barrel'].astype(int))
features_grouped['FARR_plunger'] = np.sign(features_grouped['FARR_plunger'].astype(int))
features_grouped['Tubing_anchored'] = np.sign(features_grouped['Tubing_anchored'].astype(int))
features_grouped['Setting_depth'] = features_grouped['ABSETZTEUFE']
features_grouped['Pump_bore'] = features_grouped['API_KURZBEZ_TEIL1'].astype(int)
features_grouped['Stroke_length'] = features_grouped['HUBLAENGE']
features_grouped['Pumping_speed'] = features_grouped['HUBZAHL']
features_grouped['DLS'] = features_grouped['DOGLEG']
features_grouped['Full_flow_cages'] = np.sign(features_grouped['Full Flow Cages'])
features_grouped['Mod_valve_rod_bearing'] = np.sign(features_grouped['Modifizierte \
Kolbenstangenführung mit Insert'])
features_grouped['Gas_lock_breaker'] = np.sign(features_grouped['Gas Lock Breaker'])
features_grouped['Closed_installation'] = np.sign(features_grouped['geschlossene Installation'].astype(int))
features_grouped['Collet_valve_rod_bushing'] = np.sign(features_grouped['Collet/Lock Type Valve Rod Bushing'])


features_grouped['FARR_plunger'] = features_grouped['FARR_plunger'].fillna(0)

features_grouped = features_grouped.drop(columns =['Conslot Filter', 'Stanley Filter',
                                                  'neu', 'gebr.', 'weiß', 'gelb',
                                                   'Full Flow Cages', 'Gas Lock Breaker',
                                                   'geschlossene Installation',
                                                   'Collet/Lock Type Valve Rod Bushing',
                                                   'Modifizierte Kolbenstangenführung mit Insert',
                                                  'Spiral Rod Guide 2 7/8"', 'Spiral Rod Guide 3 1/2"',
                                                  'RHAC', 'RHBM', 'RWAC', 'TH', 'DOGLEG',
                                                   'Mantelrohr 2 7/8"', 'Mantelrohr 3 1/2"',
                                                  'La Bomba Memory Gauge', 'Gasanker',
                                                   'Gasseparator - PCM', 'A4 Ventilkugeln',
                                                  'A6 Ventilkugeln','Messingbarrel', 'ABSETZTEUFE',
                                                  'API_KURZBEZ_TEIL1', 'HUBLAENGE', 'HUBZAHL',
                                                   'Steigrohranker', 'LSP ohne Gummi',
                                                   'Heavy_cages_balls',
                                                   'TP mit FARR Pl., Messing-Barrel und schweren Kugeln & Sitzen ausstatten',
                                                  '!!OLD!!DONT USE!! TP mit schweren Kugeln und Sitzen ausstatten'])

features_grouped = features_grouped.drop_duplicates()

binary_features = ['Desander','Tubing_pump','Sand_filter','New_tubing','Spiral_rod_guides',
                   'Heavy_wall_barrel','Jacket_pipe','Gas_separator','Brass_barrel',
                   'FARR_plunger','Tubing_anchored', 'A4_valve_balls',
                   'A6_valve_balls','Full_flow_cages','Mod_valve_rod_bearing','Gas_lock_breaker',
                   'Closed_installation', 'Collet_valve_rod_bushing']

continuous_features = ['DLS','Setting_depth','Pumping_speed','Stroke_length']

units = {'DLS':'[°/30m]','Setting_depth':'[m]','Pumping_speed':'[spm]','Stroke_length':'[m]',
         'Pump_bore':'[mm]'}
         
categorical_features = ['Pump_bore']

features_grouped

In [ ]:
ax = features_grouped.plot.scatter(x='DLS', y='RUNLIFE')
plt.xlabel('DLS [°/30m]')
plt.ylabel('Runlife [d]')
plt.show()

In [ ]:
features_grouped = sqldf("SELECT * FROM features_grouped WHERE DLS < 25")

In [ ]:
for i in range(len(binary_features)):
    ax = sns.boxplot(x=binary_features[i],y='RUNLIFE',data=features_grouped, width = 0.5, color = 'tab:blue')
    plt.show()


In [ ]:
for i in range(len(categorical_features)):
    ax = sns.boxplot(x=categorical_features[i],y='RUNLIFE',data=features_grouped, width = 0.5, color = 'tab:blue')
    plt.xlabel(categorical_features[i] + ' ' + units[categorical_features[i]])
    plt.ylabel('Runlife [d]')
    plt.show()

In [ ]:
for i in range(len(continuous_features)):
    ax = features_grouped.plot.scatter(x=continuous_features[i], y='RUNLIFE')
    plt.xlabel(continuous_features[i] + ' ' + units[continuous_features[i]])
    plt.ylabel('Runlife [d]')
    plt.show()

In [ ]:
ax = features_grouped.plot.scatter(x='DLS', y='Setting_depth')
plt.xlabel('DLS [°/30m]')
plt.ylabel('Setting_depth [m]')
plt.show()

In [ ]:
datum = pd.to_datetime(data_deferments['DATUM_1'], format="%m.%d.%Y %H:%M:%S")
pumpdurchmesser = data_deferments['API_KURZBEZ_TEIL1'].str[3:]
pumpentyp = pd.get_dummies(data_deferments['API_KURZBEZ_TEIL2'])
pgzustand = pd.get_dummies(data_deferments['PG_TYP2'])
tubingcondition = pd.get_dummies(data_deferments['TUB_TYP2'])
gasanker = pd.get_dummies(data_deferments['GAS_ANKER'])
packer = pd.get_dummies(data_deferments['PACK_ANKER'])
hubzahl = data_deferments['HUBZAHL']
hublaenge = data_deferments['HUBLAENGE']
farr = data_deferments['SPEZ_FARR'].fillna(0).astype(int)
spez1 = pd.get_dummies(data_deferments['SPEZ_EQUI1'])
spez2 = pd.get_dummies(data_deferments['SPEZ_EQUI2'])
spez3 = pd.get_dummies(data_deferments['SPEZ_EQUI3'])
spez4 = pd.get_dummies(data_deferments['SPEZ_EQUI4'])
spez5 = pd.get_dummies(data_deferments['SPEZ_EQUI5'])
spez6 = pd.get_dummies(data_deferments['SPEZ_EQUI6'])

workdata = pd.concat([data_deferments['SONDENKURZNAME_1'],
                      data_deferments['ABSTELLGRUND'], datum,
                      data_deferments['RUNLIFE'],data_deferments['DOGLEG'],
                      data_deferments['ABSETZTEUFE'],data_deferments['HUBZAHL'],
                      data_deferments['HUBLAENGE'], data_deferments['LEVEL3'],
                      data_deferments['LEVEL3A'], data_deferments['LEVEL4'],
                      data_deferments['LEVEL5'], hublaenge, pumpdurchmesser,
                      pumpentyp, pgzustand, tubingcondition, gasanker, packer,
                      spez1, spez2, spez3, spez4, spez5, spez6, tubingcondition], axis = 1)

features_grouped = workdata.groupby(workdata.columns, axis=1).sum()

farr_heavycage_brass = features_grouped['TP mit FARR Pl., Messing-Barrel und schweren Kugeln & Sitzen ausstatten']
heavycage = features_grouped['!!OLD!!DONT USE!! TP mit schweren Kugeln und Sitzen ausstatten']

features_grouped['Tubing_pump'] = features_grouped['TH']
features_grouped['Sand_filter'] = features_grouped['Conslot Filter'] + features_grouped['Stanley Filter']
features_grouped['New_tubing'] = features_grouped['neu'] + features_grouped['gelb']
features_grouped['Spiral_rod_guides'] = features_grouped['Spiral Rod Guide 2 7/8"'] + features_grouped['Spiral Rod Guide 3 1/2"']
features_grouped['Heavy_wall_barrel'] = features_grouped['RHAC'] + features_grouped['RHBM'] + features_grouped['TH']
features_grouped['Jacket_pipe'] = features_grouped['Mantelrohr 2 7/8"'] + features_grouped['Mantelrohr 3 1/2"']
features_grouped['Gas_separator'] = features_grouped['Gasanker'] + features_grouped['Gasseparator - PCM']
features_grouped['Brass_barrel'] = features_grouped['Messingbarrel'] + farr_heavycage_brass
features_grouped['FARR_plunger'] = farr + farr_heavycage_brass
features_grouped['Heavy_cages_balls'] = heavycage + farr_heavycage_brass
features_grouped['Tubing_anchored'] = features_grouped['Steigrohranker'] + features_grouped['LSP ohne Gummi']
features_grouped['A4_valve_balls'] = np.sign(features_grouped['A4 Ventilkugeln'] + \
                                             features_grouped['Heavy_cages_balls'])

features_grouped['A4_valve_balls'] = np.sign(features_grouped['A4 Ventilkugeln'])
features_grouped['A4_valve_balls'] = np.sign(features_grouped['A4 Ventilkugeln'])
features_grouped['A6_valve_balls'] = np.sign(features_grouped['A6 Ventilkugeln'])
features_grouped['Sand_filter'] = np.sign(features_grouped['Sand_filter'].astype(int))
features_grouped['New_tubing'] = np.sign(features_grouped['New_tubing'].astype(int))
features_grouped['Spiral_rod_guides'] = np.sign(features_grouped['Spiral_rod_guides'].astype(int))
features_grouped['Heavy_wall_barrel'] = np.sign(features_grouped['Heavy_wall_barrel'].astype(int))
features_grouped['Gas_separator'] = np.sign(features_grouped['Gas_separator'].astype(int))
features_grouped['Brass_barrel'] = np.sign(features_grouped['Brass_barrel'].astype(int))
features_grouped['FARR_plunger'] = np.sign(features_grouped['FARR_plunger'].astype(int))
features_grouped['Tubing_anchored'] = np.sign(features_grouped['Tubing_anchored'].astype(int))
features_grouped['Setting_depth'] = features_grouped['ABSETZTEUFE']
features_grouped['Pump_bore'] = features_grouped['API_KURZBEZ_TEIL1'].astype(int)
features_grouped['Stroke_length'] = features_grouped['HUBLAENGE']
features_grouped['Pumping_speed'] = features_grouped['HUBZAHL']
features_grouped['DLS'] = features_grouped['DOGLEG']
features_grouped['Full_flow_cages'] = np.sign(features_grouped['Full Flow Cages'].astype(int))
features_grouped['Mod_valve_rod_bearing'] = np.sign(features_grouped['Modifizierte \
Kolbenstangenführung mit Insert'].astype(int))
features_grouped['Gas_lock_breaker'] = np.sign(features_grouped['Gas Lock Breaker'].astype(int))
features_grouped['Closed_installation'] = np.sign(features_grouped['geschlossene Installation'].astype(int))
features_grouped['Collet_valve_rod_bushing'] = np.sign(features_grouped['Collet/Lock Type Valve Rod Bushing'].astype(int))


features_grouped['FARR_plunger'] = features_grouped['FARR_plunger'].fillna(0)

features_grouped = features_grouped.drop(columns =['Conslot Filter', 'Stanley Filter',
                                                  'neu', 'gebr.', 'weiß', 'gelb',
                                                   'Full Flow Cages', 'Gas Lock Breaker',
                                                   'geschlossene Installation',
                                                   'Collet/Lock Type Valve Rod Bushing',
                                                   'Modifizierte Kolbenstangenführung mit Insert',
                                                  'Spiral Rod Guide 2 7/8"', 'Spiral Rod Guide 3 1/2"',
                                                  'RHAC', 'RHBM', 'RWAC', 'TH', 'DOGLEG',
                                                   'Mantelrohr 2 7/8"', 'Mantelrohr 3 1/2"',
                                                  'La Bomba Memory Gauge', 'Gasanker',
                                                   'Gasseparator - PCM', 'A4 Ventilkugeln',
                                                  'A6 Ventilkugeln','Messingbarrel', 'ABSETZTEUFE',
                                                  'API_KURZBEZ_TEIL1', 'HUBLAENGE', 'HUBZAHL',
                                                   'Steigrohranker', 'LSP ohne Gummi',
                                                   'Heavy_cages_balls',
                                                   'TP mit FARR Pl., Messing-Barrel und schweren Kugeln & Sitzen ausstatten',
                                                  '!!OLD!!DONT USE!! TP mit schweren Kugeln und Sitzen ausstatten'])

features_grouped = features_grouped.drop_duplicates()

features_pgbruch = sqldf("SELECT * FROM features_grouped WHERE abstellgrund LIKE '%Pumpgestängebruch'")
features_tpfehler = sqldf("SELECT * FROM features_grouped WHERE abstellgrund LIKE '%Tiefpumpenfehler'")
features_undicht = sqldf("SELECT * FROM features_grouped WHERE abstellgrund LIKE '%Installation undicht'")

binary_features = ['Desander','Tubing_pump','Sand_filter','New_tubing','Spiral_rod_guides',
                   'Heavy_wall_barrel','Jacket_pipe','Gas_separator','Brass_barrel',
                   'FARR_plunger','Tubing_anchored', 'A4_valve_balls',
                   'A6_valve_balls','Full_flow_cages','Mod_valve_rod_bearing','Gas_lock_breaker',
                   'Closed_installation', 'Collet_valve_rod_bushing']

continuous_features = ['DLS','Setting_depth','Pumping_speed','Stroke_length']

categorical_features = ['Pump_bore']

features_grouped

In [ ]:
sqldf("SELECT * FROM features_grouped WHERE DLS > 10")

In [ ]:
for i in range(len(categorical_features)):
    ax = sns.boxplot(x=categorical_features[i],y='RUNLIFE',data=features_grouped, width = 0.5, color = 'tab:blue')
    plt.xlabel(categorical_features[i] + ' ' + units[categorical_features[i]])
    plt.ylabel('Runlife [d]')
    plt.show()

In [ ]:
for i in range(len(binary_features)):
    ax = sns.boxplot(x=binary_features[i],y='RUNLIFE',data=features_pgbruch, width = 0.5, color = 'tab:blue')
    plt.show()

In [ ]:
for i in range(len(continuous_features)):
    ax = features_pgbruch.plot.scatter(x=continuous_features[i], y='RUNLIFE')
    plt.xlabel(continuous_features[i] + ' ' + units[continuous_features[i]])
    plt.ylabel('Runlife [d]')
    plt.show()

In [ ]:
for i in range(len(categorical_features)):
    ax = sns.boxplot(x=categorical_features[i],y='RUNLIFE',data=features_pgbruch, width = 0.5, color = 'tab:blue')
    plt.xlabel(categorical_features[i] + ' ' + units[categorical_features[i]])
    plt.ylabel('Runlife [d]')
    plt.show()

In [ ]:
for i in range(len(binary_features)):
    ax = sns.boxplot(x=binary_features[i],y='RUNLIFE',data=features_tpfehler, width = 0.5, color = 'tab:blue')
    plt.show()

In [ ]:
for i in range(len(continuous_features)):
    ax = features_tpfehler.plot.scatter(x=continuous_features[i], y='RUNLIFE')
    plt.xlabel(continuous_features[i] + ' ' + units[continuous_features[i]])
    plt.ylabel('Runlife [d]')
    plt.show()

In [ ]:
for i in range(len(categorical_features)):
    ax = sns.boxplot(x=categorical_features[i],y='RUNLIFE',data=features_tpfehler, width = 0.5, color = 'tab:blue')
    plt.xlabel(categorical_features[i] + ' ' + units[categorical_features[i]])
    plt.ylabel('Runlife [d]')
    plt.show()

In [ ]:
for i in range(len(binary_features)):
    ax = sns.boxplot(x=binary_features[i],y='RUNLIFE',data=features_undicht, width = 0.5, color = 'tab:blue')
    plt.show()

In [ ]:
for i in range(len(continuous_features)):
    ax = features_undicht.plot.scatter(x=continuous_features[i], y='RUNLIFE')
    plt.show()

In [ ]:
for i in range(len(categorical_features)):
    ax = sns.boxplot(x=categorical_features[i],y='RUNLIFE',data=features_tpfehler, width = 0.5, color = 'tab:blue')
    plt.xlabel(categorical_features[i] + ' ' + units[categorical_features[i]])
    plt.ylabel('Runlife [d]')
    plt.show()

In [ ]:
df = (features_grouped.copy())

df['DATUM_1'] =  pd.to_datetime(df['DATUM_1'], format="%m.%d.%Y %H:%M:%S")

df = df.sort_values(['SONDENKURZNAME_1', 'DATUM_1'], ascending = [True, True])

df = df.reset_index(drop = True)

components = ['Desander','Sand_filter','Spiral_rod_guides',
             'Jacket_pipe','Gas_separator','Brass_barrel',
              'A4_valve_balls','FARR_plunger','Full_flow_cages',
              'Mod_valve_rod_bearing','Gas_lock_breaker', 
              'Collet_valve_rod_bushing', 'Jacket_pipe']

rca_levels = ['ABSTELLGRUND','LEVEL3','LEVEL3A','LEVEL4','LEVEL5']

 
def installment_check(df,components, rca_levels):
    
    df['Trend'] = np.nan
    df['Delta_runlife'] = np.nan
    df['Delta_depth'] = np.nan
    df['Delta_DLS'] = np.nan
    df['Delta_Pumping_speed'] = np.nan
    df['Different_Abstellgrund'] = np.nan
    
    for i in range (len(components)):
        column_name = 'Installment_' + components[i]
        df[column_name] = np.nan

    for i in range (len(rca_levels)):
        column_name = 'Previous_' + rca_levels[i]
        df[column_name] = np.nan
    
    for i in range(len(df)-1):
        if df.at[i, 'SONDENKURZNAME_1'] == df.at[i+1, 'SONDENKURZNAME_1']:
            
            for j in range(len(components)):
                if df.at[i, components[j]] > df.at[i+1, components[j]]:
                    df.loc[i+1,['Installment_' + components[j]]] = 'Component removed'
                    
                if df.at[i, components[j]] < df.at[i+1, components[j]]:
                    df.loc[i+1,['Installment_' + components[j]]] = 'Component installed'

                if df.at[i, components[j]] == df.at[i+1, components[j]]:
                    df.loc[i+1,['Installment_' + components[j]]] = 'Same design'
                    
            for j in range(len(rca_levels)):
                    df.loc[i+1,['Previous_' + rca_levels[j]]] =  \
                    df.loc[i,rca_levels[j]]

            df.loc[i+1,['Delta_runlife']] = df.at[i+1, 'RUNLIFE'] - df.at[i, 'RUNLIFE']
            df.loc[i+1,['Delta_depth']] = df.at[i+1, 'Setting_depth'] - df.at[i, 'Setting_depth']
            df.loc[i+1,['Delta_DLS']] = df.at[i+1, 'DLS'] - df.at[i, 'DLS']
            df.loc[i+1,['Delta_Pumping_speed']] = df.at[i+1, 'Pumping_speed'] - df.at[i, 'Pumping_speed']        
                    
            if df.at[i, 'RUNLIFE'] < df.at[i+1, 'RUNLIFE']:
                df.loc[i+1,['Trend']] = 'Higher'

            elif df.at[i, 'RUNLIFE'] > df.at[i+1, 'RUNLIFE']:
                df.loc[i+1,['Trend']] = 'Lower'
                
            if df.at[i, 'ABSTELLGRUND'] == df.at[i+1, 'ABSTELLGRUND']:
                df.loc[i+1,['Different_Abstellgrund']] = False

            else:
                df.loc[i+1,['Different_Abstellgrund']] = True
            
    return df

installment_check(df,components,rca_levels)

In [ ]:
installment_ffc = sqldf("SELECT sondenkurzname_1 AS Name, abstellgrund, runlife, delta_runlife, \
Different_abstellgrund, level3, \
level3A, level4, level5, previous_abstellgrund, previous_level3, previous_level3a, previous_level4,\
previous_level5  \
FROM df WHERE [Installment_full_flow_cages] LIKE \
'%Component installed%'") 

print(installment_ffc['Delta_runlife'].mean())
print(installment_ffc['Different_Abstellgrund'].sum())

installment_ffc

In [ ]:
installment_farr = sqldf("SELECT sondenkurzname_1, abstellgrund, runlife, delta_runlife, \
Different_abstellgrund, datum_1, level3, \
level3A, level4, level5, previous_abstellgrund, previous_level3, previous_level3a, previous_level4,\
previous_level5  \
FROM df WHERE Installment_FARR_plunger LIKE \
'%Component installed%'") 

print(installment_farr['Delta_runlife'].mean())
print(installment_farr['Different_Abstellgrund'].sum())

installment_farr

In [ ]:
installment_mod_valve = sqldf("SELECT sondenkurzname_1, abstellgrund, runlife as RL, delta_runlife, \
Different_abstellgrund, level3, \
level3A, level4, level5, previous_abstellgrund, previous_level3, previous_level3a, previous_level4,\
previous_level5  \
FROM df WHERE [Installment_mod_valve_rod_bearing] LIKE \
'%Component installed%'") 

print(installment_mod_valve['Delta_runlife'].mean())
print(installment_mod_valve['Different_Abstellgrund'].sum())

installment_mod_valve

In [ ]:
installment_farr_plunger = sqldf("SELECT sondenkurzname_1, abstellgrund, runlife as RL, delta_runlife, \
Different_abstellgrund, level3, \
level3A, level4, level5, previous_abstellgrund, previous_level3, previous_level3a, previous_level4,\
previous_level5  \
FROM df WHERE [Installment_FARR_plunger] LIKE \
'%Component installed%'") 

print(installment_farr_plunger['Delta_runlife'].mean())
print(installment_farr_plunger['Different_Abstellgrund'].sum())

installment_farr_plunger

In [ ]:
installment_sprg = sqldf("SELECT sondenkurzname_1, abstellgrund, runlife as RL, delta_runlife, \
Different_abstellgrund, level3, \
level3A, level4, level5, previous_abstellgrund, previous_level3, previous_level3a, previous_level4,\
previous_level5  \
FROM df WHERE [Installment_Spiral_rod_guides] LIKE \
'%Component installed%'") 

print(installment_sprg['Delta_runlife'].mean())
print(installment_sprg['Different_Abstellgrund'].sum())

installment_sprg

In [ ]:
installment_sprg = sqldf("SELECT sondenkurzname_1, abstellgrund, runlife as RL, delta_runlife, \
Different_abstellgrund, level3, \
level3A, level4, level5, previous_abstellgrund, previous_level3, previous_level3a, previous_level4,\
previous_level5  \
FROM df WHERE [Installment_Spiral_rod_guides] LIKE '%Component installed%' \
AND previous_abstellgrund LIKE '%Tiefpumpenfehler%'") 

print(installment_sprg['Delta_runlife'].mean())
print(installment_sprg['Different_Abstellgrund'].sum())

installment_sprg

In [ ]:
installment_brass = sqldf("SELECT sondenkurzname_1, abstellgrund, runlife as RL, delta_runlife, \
Different_abstellgrund, level3, \
level3A, level4, level5, previous_abstellgrund, previous_level3, previous_level3a, previous_level4,\
previous_level5  \
FROM df WHERE [Installment_Brass_barrel] LIKE '%Component installed%' ")

print(installment_brass['Delta_runlife'].mean())
print(installment_brass['Different_Abstellgrund'].sum())

installment_brass

In [ ]:
installment_jacket = sqldf("SELECT sondenkurzname_1, abstellgrund, runlife as RL, delta_runlife, \
Different_abstellgrund, level3, \
level3A, level4, level5, previous_abstellgrund, previous_level3, previous_level3a, previous_level4,\
previous_level5  \
FROM df WHERE [Installment_Jacket_pipe] LIKE '%Component installed%'") 

print(installment_jacket['Delta_runlife'].mean())
print(installment_jacket['Different_Abstellgrund'].sum())

installment_jacket

In [ ]:
installment_glb = sqldf("SELECT sondenkurzname_1, abstellgrund, runlife as RL, delta_runlife, \
Different_abstellgrund, level3, \
level3A, level4, level5, previous_abstellgrund, previous_level3, previous_level3a, previous_level4,\
previous_level5  \
FROM df WHERE [Installment_Gas_lock_breaker] LIKE '%Component installed%'") 

print(installment_glb['Delta_runlife'].mean())
print(installment_glb['Different_Abstellgrund'].sum())

installment_glb

In [ ]:
installment_gas_anchors = sqldf("SELECT sondenkurzname_1, abstellgrund, runlife as RL, delta_runlife, \
Different_abstellgrund, level3, \
level3A, level4, level5, previous_abstellgrund, previous_level3, previous_level3a, previous_level4,\
previous_level5  \
FROM df WHERE [Installment_Gas_separator] LIKE '%Component installed%'") 

print(installment_gas_anchors['Delta_runlife'].mean())
print(installment_gas_anchors['Different_Abstellgrund'].sum())

installment_gas_anchors

In [ ]:
installment_desander = sqldf("SELECT sondenkurzname_1, abstellgrund, runlife as RL, delta_runlife, \
Different_abstellgrund, level3, \
level3A, level4, level5, previous_abstellgrund, previous_level3, previous_level3a, previous_level4,\
previous_level5  \
FROM df WHERE [Installment_Desander] LIKE '%Component installed%'") 

print(installment_desander['Delta_runlife'].mean())
print(installment_desander['Different_Abstellgrund'].sum())

installment_desander

In [ ]:
installment_collet = sqldf("SELECT sondenkurzname_1, abstellgrund, runlife as RL, delta_runlife, \
Different_abstellgrund, level3, \
level3A, level4, level5, previous_abstellgrund, previous_level3, previous_level3a, previous_level4,\
previous_level5  \
FROM df WHERE [Installment_Collet_valve_rod_bushing] LIKE '%Component installed%'") 

print(installment_collet['Delta_runlife'].mean())
print(installment_collet['Different_Abstellgrund'].sum())

installment_desander

In [ ]:
installment_srg_pgbruch = sqldf("SELECT * FROM df WHERE [Installment_Spiral_rod_guides] IS \
'Component installed' AND previous_level3a LIKE 'Valve rod'")


print(f"Mean Delta_runlife = {installment_srg_pgbruch['Delta_runlife'].mean()}")

installment_srg_pgbruch

In [ ]:
installment_srg_pgbruch = sqldf("SELECT * FROM df WHERE [Installment_FARR_plunger] IS \
'Component installed'")


print(f"Mean Delta_runlife = {installment_srg_pgbruch['Delta_runlife'].mean()}")

installment_srg_pgbruch

In [ ]:
installment_srg_pgbruch = sqldf("SELECT * FROM df WHERE [Installment_Spiral_rod_guides] IS \
'Component installed' AND previous_abstellgrund LIKE '%Pumpgestängebruch%'")


print(f"Mean Delta_runlife = {installment_srg_pgbruch['Delta_runlife'].mean()}")

installment_srg_pgbruch

In [ ]:
def scatter_hist(x, y, ax, ax_histx, ax_histy, xlabel, ylabel):
    # no labels
    ax_histx.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis="y", labelleft=False)

    # the scatter plot:
    ax.scatter(x, y)

    ax_histx.hist(x, bins='scott')
    ax_histy.hist(y, bins='scott', orientation='horizontal')
    
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    
def delta_plot(array, x_input, y_input, xlabel, ylabel):
    
    x = array[:,x_input]
    y = array[:,y_input]
    
    plt.scatter(x,y)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()
    
    fig = plt.figure(figsize=(6, 6))

    gs = fig.add_gridspec(2, 2,  width_ratios=(4, 1), height_ratios=(1, 4),
                          left=0.1, right=0.9, bottom=0.1, top=0.9,
                          wspace=0.05, hspace=0.05)

    ax = fig.add_subplot(gs[1, 0])
    ax_histx = fig.add_subplot(gs[0, 0], sharex=ax)
    ax_histy = fig.add_subplot(gs[1, 1], sharey=ax)


    scatter_hist(array[:,x_input], array[:,y_input], ax, ax_histx, ax_histy, xlabel, ylabel)

In [ ]:
df = df.dropna(subset = ('Delta_DLS', 'Delta_depth', 'Delta_Pumping_speed','Delta_runlife'))
df_undicht = sqldf("SELECT * FROM df WHERE Previous_ABSTELLGRUND LIKE '%Installation undicht%'")
df_pgbruch = sqldf("SELECT * FROM df WHERE Previous_ABSTELLGRUND LIKE '%Pumpgestängebruch%'")
df_tpfehler = sqldf("SELECT * FROM df WHERE Previous_ABSTELLGRUND LIKE '%Tiefpumpenfehler%'")

array = df[['Delta_DLS', 'Delta_depth', 'Delta_Pumping_speed', 'Delta_runlife']].to_numpy()
array_undicht = df_undicht[['Delta_DLS', 'Delta_depth', 'Delta_Pumping_speed', 'Delta_runlife']].to_numpy()
array_pgbruch = df_pgbruch[['Delta_DLS', 'Delta_depth', 'Delta_Pumping_speed', 'Delta_runlife']].to_numpy()
array_tpfehler = df_tpfehler[['Delta_DLS', 'Delta_depth', 'Delta_Pumping_speed','Delta_runlife']].to_numpy()

In [ ]:
delta_plot(array, 0, 3, 'Delta DLS [°/30m]', 'Delta runlife [d]')

In [ ]:
delta_plot(array, 1, 3, 'Delta setting depth [m]', 'Delta runlife [d]')

In [ ]:
delta_plot(array, 2, 3, 'Delta pumping speed [spm]', 'Delta runlife [d]')

In [ ]:
delta_plot(array_undicht, 0, 3, 'Delta DLS [°/30m]', 'Delta runlife [d]')

In [ ]:
delta_plot(array_undicht, 1, 3, 'Delta setting depth [m]', 'Delta runlife [d]')

In [ ]:
delta_plot(array_undicht, 2, 3, 'Delta pumping speed [spm]', 'Delta runlife [d]')

In [ ]:
delta_plot(array_pgbruch, 0, 3, 'Delta DLS [°/30m]', 'Delta runlife [d]')

In [ ]:
delta_plot(array_pgbruch, 1, 3, 'Delta setting depth [m]', 'Delta runlife [d]')

In [ ]:
delta_plot(array_pgbruch, 2, 3, 'Delta pumping speed [spm]', 'Delta runlife [d]')

In [ ]:
delta_plot(array_tpfehler, 0, 3, 'Delta DLS [°/30m]', 'Delta runlife [d]')

In [ ]:
delta_plot(array_tpfehler, 1, 3, 'Delta setting depth [m]', 'Delta runlife [d]')

In [ ]:
delta_plot(array_tpfehler, 2, 3, 'Delta pumping speed [spm]', 'Delta runlife [d]')